In [5]:
# RERUN AT HOME
import glob

In [6]:
# RERUN AT HOME
path = "/Users/jasur/desktop/FER13/train" # use your path, all images

In [7]:
# RERUN AT HOME
angry_images = glob.glob(path + '/angry/*', recursive=True)
fear_images = glob.glob(path + '/fear/*', recursive=True)
happy_images = glob.glob(path + '/happy/*', recursive=True)
neutral_images = glob.glob(path + '/neutral/*', recursive=True)
surprise_images = glob.glob(path + '/surprise/*', recursive=True)
sad_images = glob.glob(path + '/sad/*', recursive=True)

In [8]:
train_images = angry_images[:1400] + fear_images[:1400]+ happy_images[:1400] + neutral_images[:1400] + surprise_images[:1400] + sad_images[:1400]
print("Number of images: ", len(train_images))

Number of images:  8400


In [9]:
validation_images = angry_images[1400:1600] + fear_images[1400:1600] + happy_images[1400:1600] + neutral_images[1400:1600] + surprise_images[1400:1600] + sad_images[1400:1600]
print("Number of images: ", len(validation_images))
test_images = angry_images[1600:1800] + fear_images[1600:1800] + happy_images[1600:1800] + neutral_images[1600:1800] + surprise_images[1600:1800] + sad_images[1600:1800]
print("Number of images: ", len(test_images))

Number of images:  1200
Number of images:  1200


In [10]:
from PIL import Image
import os
import numpy as np
import pickle
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Assuming you have a list of image paths like this
image_paths = train_images

# Set the target size for resizing the images
target_size = (224, 224)

# Create empty lists to store the images and labels
images = []
labels = []

def preprocess(image_paths, cat, train = False, label_encoder = None):
    images, labels = [], []

    for path in image_paths:
        img = Image.open(path).convert('RGB')
        img = img.resize(target_size)
        img_array = np.array(img) / 255.0  
        images.append(img_array)

    for path in image_paths:
        label = os.path.basename(os.path.dirname(path))
        labels.append(label)
    if train:
        label_encoder = LabelEncoder()
        encoded_labels = label_encoder.fit_transform(labels)
    else:
        encoded_labels = label_encoder.transform(labels)

    onehot_encoder = OneHotEncoder(sparse=False)
    encoded_labels = encoded_labels.reshape(len(encoded_labels), 1)
    onehot_labels = onehot_encoder.fit_transform(encoded_labels)

    images = np.array([img.reshape(target_size[0], target_size[1], 3) for img in images])
    labels = np.array(onehot_labels)

    dataset = tf.data.Dataset.from_tensor_slices((images, labels))

    batch_size = 32
    shuffle_buffer_size = len(images) 

    dataset = dataset.shuffle(buffer_size=shuffle_buffer_size)
    dataset = dataset.batch(batch_size)

    save_path = "/Users/jasur/desktop/FER13/V2/"
    os.makedirs(save_path, exist_ok=True)

    np.save(os.path.join(save_path, 'X_' + cat + '.npy'), images)
    np.save(os.path.join(save_path, 'y_' + cat + '.npy'), labels)

    return dataset, label_encoder


train_dataset, label_encoder = preprocess(train_images, 'train', train=True)
validation_dataset, _ = preprocess(validation_images, 'validation',label_encoder=label_encoder)
test_dataset, _ = preprocess(test_images, 'test', label_encoder=label_encoder)

/Users/jasur/Library/Python/3.9/lib/python/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/jasur/Library/Python/3.9/lib/python/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/jasur/Library/Python/3.9/lib/python/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [1]:
from PIL import Image
import os
import numpy as np
import pickle
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

save_path = "/Users/jasur/desktop/FER13/Data_Even_More_.np_arrays"

images = np.load(os.path.join(save_path, 'X_train.npy'))
labels = np.load(os.path.join(save_path, 'y_train.npy'))
train_dataset = tf.data.Dataset.from_tensor_slices((images, labels))

batch_size = 32
shuffle_buffer_size = len(images) 

train_dataset = train_dataset.shuffle(buffer_size=shuffle_buffer_size)
train_dataset = train_dataset.batch(batch_size)

In [2]:
images = np.load(os.path.join(save_path, 'X_val.npy'))
labels = np.load(os.path.join(save_path, 'y_val.npy'))
validation_dataset = tf.data.Dataset.from_tensor_slices((images, labels))

batch_size = 32
shuffle_buffer_size = len(images) 

validation_dataset = validation_dataset.shuffle(buffer_size=shuffle_buffer_size)
validation_dataset = validation_dataset.batch(batch_size)

In [3]:
images = np.load(os.path.join(save_path, 'X_test.npy'))
labels = np.load(os.path.join(save_path, 'y_test.npy'))
test_dataset = tf.data.Dataset.from_tensor_slices((images, labels))

batch_size = 32
shuffle_buffer_size = len(images) 

test_dataset = test_dataset.shuffle(buffer_size=shuffle_buffer_size)
test_dataset = test_dataset.batch(batch_size)

In [10]:
from tensorflow import keras
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout,\
BatchNormalization, Activation, SpatialDropout2D, GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=(224, 224, 3)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(SpatialDropout2D(0.2))

model.add(Conv2D(128, (3, 3), padding='same', input_shape=(224, 224, 3)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(SpatialDropout2D(0.2))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.35))
model.add(Dense(6, activation='softmax'))

model.compile(optimizer=Adam(), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])


In [11]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.vis_utils import plot_model

from sklearn.metrics import classification_report, confusion_matrix

chk_path = 'CustomNet.h5'

checkpoint = ModelCheckpoint(filepath=chk_path,
                             save_best_only=True,
                             verbose=1,
                             moniter='val_accuracy')

earlystop = EarlyStopping(monitor='val_accuracy',  
                          patience=3, 
                          verbose=1, 
                          restore_best_weights=True)

callbacks = [checkpoint, earlystop]

In [13]:
model_sum = model.fit(train_dataset, validation_data=validation_dataset, epochs=3, callbacks=callbacks)

Epoch 1/3


263/263 [==============================] - ETA: 0s - loss: 1.8222 - accuracy: 0.1756
Epoch 1: val_loss improved from inf to 1.79068, saving model to CustomNet.h5
263/263 [==============================] - 1394s 5s/step - loss: 1.8222 - accuracy: 0.1756 - val_loss: 1.7907 - val_accuracy: 0.1883
Epoch 2/3
263/263 [==============================] - ETA: 0s - loss: 1.7925 - accuracy: 0.1786
Epoch 2: val_loss did not improve from 1.79068
263/263 [==============================] - 1423s 5s/step - loss: 1.7925 - accuracy: 0.1786 - val_loss: 1.7911 - val_accuracy: 0.1958
Epoch 3/3
263/263 [==============================] - ETA: 0s - loss: 1.7847 - accuracy: 0.2024
Epoch 3: val_loss improved from 1.79068 to 1.79023, saving model to CustomNet.h5
263/263 [==============================] - 1396s 5s/step - loss: 1.7847 - accuracy: 0.2024 - val_loss: 1.7902 - val_accuracy: 0.1992


In [14]:
model_sum = model.fit(train_dataset, validation_data=validation_dataset, epochs=3, callbacks=callbacks)

Epoch 1/3
263/263 [==============================] - ETA: 0s - loss: 1.7776 - accuracy: 0.2083
Epoch 1: val_loss improved from 1.79023 to 1.78301, saving model to CustomNet.h5
263/263 [==============================] - 1450s 6s/step - loss: 1.7776 - accuracy: 0.2083 - val_loss: 1.7830 - val_accuracy: 0.1908
Epoch 2/3
263/263 [==============================] - ETA: 0s - loss: 1.7479 - accuracy: 0.2185
Epoch 2: val_loss did not improve from 1.78301
263/263 [==============================] - 1606s 6s/step - loss: 1.7479 - accuracy: 0.2185 - val_loss: 1.8134 - val_accuracy: 0.1900
Epoch 3/3
263/263 [==============================] - ETA: 0s - loss: 1.7469 - accuracy: 0.2268
Epoch 3: val_loss did not improve from 1.78301
263/263 [==============================] - 1767s 7s/step - loss: 1.7469 - accuracy: 0.2268 - val_loss: 1.8128 - val_accuracy: 0.2017


In [76]:
from tensorflow import keras
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, \
    BatchNormalization, Activation, SpatialDropout2D, GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras.models import Sequential, Model


model_1 = keras.applications.resnet.ResNet50(input_shape=(224, 224, 3), weights = 'imagenet', include_top = False)

model_1.trainable = False

model_2 = Sequential()

model_2.add(Conv2D(128, (5, 5), padding='same', input_shape=model_1.output_shape[1:]))
model_2.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
model_2.add(BatchNormalization())
model_2.add(MaxPooling2D(pool_size=(2, 2)))
model_2.add(SpatialDropout2D(0.25))

model_2.add(Flatten())

model_2.add(Dense(64, activation='relu'))
model_2.add(Dense(256, activation='relu'))
model_2.add(Dense(6, activation='softmax'))

model = Model(inputs=model_1.input, outputs=model_2(model_1.output))

for layer in model.layers[:100]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=0.0001,
                  decay=1e-6), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

In [70]:
model.layers[-1].summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_44 (Conv2D)          (None, 7, 7, 128)         6553728   
                                                                 
 conv2d_45 (Conv2D)          (None, 7, 7, 256)         295168    
                                                                 
 batch_normalization_11 (Bat  (None, 7, 7, 256)        1024      
 chNormalization)                                                
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 3, 3, 256)        0         
 g2D)                                                            
                                                                 
 spatial_dropout2d_19 (Spati  (None, 3, 3, 256)        0         
 alDropout2D)                                                    
                                                     

In [77]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

chk_path = 'CustomResNet.h5'

checkpoint = ModelCheckpoint(filepath=chk_path,
                             save_best_only=True,
                             verbose=1,
                             moniter='val_accuracy')

earlystop = EarlyStopping(monitor='val_accuracy',  
                          patience=3, 
                          verbose=1, 
                          restore_best_weights=True)

callbacks = [checkpoint, earlystop]

In [78]:
model_sum = model.fit(train_dataset, validation_data=validation_dataset, epochs=2, callbacks=callbacks)

Epoch 1/2


263/263 [==============================] - ETA: 0s - loss: 1.4329 - accuracy: 0.4301
Epoch 1: val_loss improved from inf to 1.91655, saving model to CustomResNet.h5
263/263 [==============================] - 2350s 9s/step - loss: 1.4329 - accuracy: 0.4301 - val_loss: 1.9166 - val_accuracy: 0.1667
Epoch 2/2
263/263 [==============================] - ETA: 0s - loss: 1.0995 - accuracy: 0.5740
Epoch 2: val_loss improved from 1.91655 to 1.79746, saving model to CustomResNet.h5
263/263 [==============================] - 1941s 7s/step - loss: 1.0995 - accuracy: 0.5740 - val_loss: 1.7975 - val_accuracy: 0.2117


In [79]:
model_sum = model.fit(train_dataset, validation_data=validation_dataset, epochs=10, callbacks=callbacks)

Epoch 1/10


263/263 [==============================] - ETA: 0s - loss: 0.8477 - accuracy: 0.6833
Epoch 1: val_loss improved from 1.79746 to 1.55126, saving model to CustomResNet.h5
263/263 [==============================] - 1918s 7s/step - loss: 0.8477 - accuracy: 0.6833 - val_loss: 1.5513 - val_accuracy: 0.4017
Epoch 2/10
263/263 [==============================] - ETA: 0s - loss: 0.6001 - accuracy: 0.7757
Epoch 2: val_loss did not improve from 1.55126
263/263 [==============================] - 1877s 7s/step - loss: 0.6001 - accuracy: 0.7757 - val_loss: 1.6102 - val_accuracy: 0.4508
Epoch 3/10
263/263 [==============================] - ETA: 0s - loss: 0.3650 - accuracy: 0.8676
Epoch 3: val_loss did not improve from 1.55126
263/263 [==============================] - 1914s 7s/step - loss: 0.3650 - accuracy: 0.8676 - val_loss: 1.5997 - val_accuracy: 0.5325
Epoch 4/10
263/263 [==============================] - ETA: 0s - loss: 0.2196 - accuracy: 0.9256
Epoch 4: val_loss did not improve from 1.55126
263

In [95]:
from tensorflow.keras.applications import VGG16
from tensorflow import keras
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, \
    BatchNormalization, Activation, SpatialDropout2D, GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras.models import Sequential, Model


model_1 = VGG16(weights = 'imagenet', input_shape=(224, 224, 3), include_top = False)

model_1.trainable = False


model_2 = Sequential()

model_2.add(Conv2D(128, (5, 5), padding='same', input_shape=model_1.output_shape[1:]))
model_2.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
model_2.add(BatchNormalization())
model_2.add(MaxPooling2D(pool_size=(2, 2)))
model_2.add(SpatialDropout2D(0.25))

model_2.add(Flatten())

model_2.add(Dense(64, activation='relu'))
model_2.add(Dense(256, activation='relu'))
model_2.add(Dense(6, activation='softmax'))

model = Model(inputs=model_1.input, outputs=model_2(model_1.output))

for layer in model.layers[:100]:
    layer.trainable = True
    
model.compile(optimizer=Adam(learning_rate=0.0001,
                  decay=1e-6), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

In [96]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

chk_path = 'CustomVGG.h5'

checkpoint = ModelCheckpoint(filepath=chk_path,
                             save_best_only=True,
                             verbose=1,
                             moniter='val_accuracy')

earlystop = EarlyStopping(monitor='val_accuracy',  
                          patience=3, 
                          verbose=1, 
                          restore_best_weights=True)

callbacks = [checkpoint, earlystop]

In [97]:
model_sum = model.fit(train_dataset, validation_data=validation_dataset, epochs=2, callbacks=callbacks)

Epoch 1/2


263/263 [==============================] - ETA: 0s - loss: 1.7238 - accuracy: 0.2496 
Epoch 1: val_loss improved from inf to 1.59768, saving model to CustomVGG.h5
263/263 [==============================] - 4490s 17s/step - loss: 1.7238 - accuracy: 0.2496 - val_loss: 1.5977 - val_accuracy: 0.3425
Epoch 2/2
263/263 [==============================] - ETA: 0s - loss: 1.4441 - accuracy: 0.4275 
Epoch 2: val_loss improved from 1.59768 to 1.36291, saving model to CustomVGG.h5
263/263 [==============================] - 3891s 15s/step - loss: 1.4441 - accuracy: 0.4275 - val_loss: 1.3629 - val_accuracy: 0.4708


In [98]:
model_sum = model.fit(train_dataset, validation_data=validation_dataset, epochs=7, callbacks=callbacks)

Epoch 1/7


263/263 [==============================] - ETA: 0s - loss: 1.2739 - accuracy: 0.4980 
Epoch 1: val_loss improved from 1.36291 to 1.25657, saving model to CustomVGG.h5
263/263 [==============================] - 3875s 15s/step - loss: 1.2739 - accuracy: 0.4980 - val_loss: 1.2566 - val_accuracy: 0.5075
Epoch 2/7
263/263 [==============================] - ETA: 0s - loss: 1.1708 - accuracy: 0.5432 
Epoch 2: val_loss improved from 1.25657 to 1.23136, saving model to CustomVGG.h5
263/263 [==============================] - 3859s 15s/step - loss: 1.1708 - accuracy: 0.5432 - val_loss: 1.2314 - val_accuracy: 0.5200
Epoch 3/7
263/263 [==============================] - ETA: 0s - loss: 1.0877 - accuracy: 0.5752 
Epoch 3: val_loss improved from 1.23136 to 1.14636, saving model to CustomVGG.h5
263/263 [==============================] - 3875s 15s/step - loss: 1.0877 - accuracy: 0.5752 - val_loss: 1.1464 - val_accuracy: 0.5592
Epoch 4/7
263/263 [==============================] - ETA: 0s - loss: 1.0042 -

In [88]:
for layer in model.layers:
    print(layer)
    #layer.trainable = True

In [33]:
from tensorflow.keras.applications import EfficientNetB7
from tensorflow import keras
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, \
    BatchNormalization, SpatialDropout2D
from keras.optimizers import Adam
from keras.models import Sequential, Model

# Load pre-trained EfficientNetB0 model
model_1 = EfficientNetB7(weights='imagenet', input_shape=(224, 224, 3), include_top=False)

model_1.trainable = False

model_2 = Sequential()

model_2.add(Conv2D(128, (5, 5), padding='same', input_shape=model_1.output_shape[1:]))
# model_2.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
model_2.add(BatchNormalization())
model_2.add(MaxPooling2D(pool_size=(2, 2)))
model_2.add(SpatialDropout2D(0.25))

model_2.add(Flatten())

model_2.add(Dense(64, activation='relu'))
model_2.add(Dense(256, activation='relu'))
model_2.add(Dense(6, activation='softmax'))

model = Model(inputs=model_1.input, outputs=model_2(model_1.output))

# Fine-tune only a subset of layers
for layer in model.layers[:100]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=0.0001, decay=1e-6),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [35]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

chk_path = 'CustomEfficientB7.h5'

checkpoint = ModelCheckpoint(filepath=chk_path,
                             save_best_only=True,
                             verbose=1,
                             moniter='val_accuracy')

earlystop = EarlyStopping(monitor='val_accuracy',  
                          patience=3, 
                          verbose=1, 
                          restore_best_weights=True)

callbacks = [checkpoint, earlystop]

In [32]:
model_sum = model.fit(train_dataset, validation_data=validation_dataset, epochs=15, callbacks=callbacks)

Epoch 1/15
263/263 [==============================] - ETA: 0s - loss: 1.7257 - accuracy: 0.2582
Epoch 1: val_loss did not improve from 1.73317
263/263 [==============================] - 46s 173ms/step - loss: 1.7257 - accuracy: 0.2582 - val_loss: 1.7395 - val_accuracy: 0.2767
Epoch 2/15
263/263 [==============================] - ETA: 0s - loss: 1.7127 - accuracy: 0.2790
Epoch 2: val_loss did not improve from 1.73317
263/263 [==============================] - 33s 126ms/step - loss: 1.7127 - accuracy: 0.2790 - val_loss: 1.7713 - val_accuracy: 0.2400
Epoch 3/15
263/263 [==============================] - ETA: 0s - loss: 1.7010 - accuracy: 0.2832
Epoch 3: val_loss did not improve from 1.73317
263/263 [==============================] - 38s 146ms/step - loss: 1.7010 - accuracy: 0.2832 - val_loss: 1.7639 - val_accuracy: 0.2492
Epoch 4/15
263/263 [==============================] - ETA: 0s - loss: 1.6883 - accuracy: 0.2982
Epoch 4: val_loss did not improve from 1.73317
Restoring model weights fr

In [58]:
%ls

CustomNet.h5
CustomNet_VGG_ResNet_EfficientNet_Training.ipynb
CustomVGG.h5
CustomVGG_LATEST.h5
Final_Project.ipynb
Loss_Accuracy.ipynb
More_Data_Training.ipynb
ResNet_VGG_Testing.ipynb
VGG_Face.ipynb


In [59]:
from keras.applications import VGG16
from keras.layers import MaxPooling2D

model_1 = VGG16(weights = None, input_shape=(224, 224, 3), include_top = False)

model_1.trainable = False


model_2 = Sequential()

model_2.add(Conv2D(128, (5, 5), padding='same', input_shape=model_1.output_shape[1:]))
model_2.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
model_2.add(BatchNormalization())
model_2.add(MaxPooling2D(pool_size=(2, 2)))
model_2.add(SpatialDropout2D(0.25))

model_2.add(Flatten())

model_2.add(Dense(64, activation='relu'))
model_2.add(Dense(256, activation='relu'))
model_2.add(Dense(6, activation='softmax'))

model = Model(inputs=model_1.input, outputs=model_2(model_1.output))

model.load_weights('./CustomVGG_LATEST.h5')
    
model.compile(optimizer=Adam(learning_rate=0.0001), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

In [60]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

chk_path = '../CustomVGG_LATEST.h5'

checkpoint = ModelCheckpoint(filepath=chk_path,
                             save_best_only=True,
                             verbose=1,
                             moniter='val_accuracy')

earlystop = EarlyStopping(monitor='val_accuracy',  
                          patience=3, 
                          verbose=1, 
                          restore_best_weights=True)

callbacks = [checkpoint, earlystop]

# Epoch 4
model.fit(train_dataset, validation_data=validation_dataset, epochs=1, callbacks=callbacks)

595/595 [==============================] - ETA: 0s - loss: 0.8181 - accuracy: 0.6924
Epoch 1: val_loss improved from inf to 0.83177, saving model to ../CustomVGG_LATEST.h5


/Users/jasur/Library/Python/3.9/lib/python/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


595/595 [==============================] - 3195s 5s/step - loss: 0.8181 - accuracy: 0.6924 - val_loss: 0.8318 - val_accuracy: 0.6850


In [53]:
# Epoch 2 and 3
model.fit(train_dataset, validation_data=validation_dataset, epochs=2, callbacks=callbacks)

Epoch 1/2
595/595 [==============================] - ETA: 0s - loss: 0.8665 - accuracy: 0.6772
Epoch 1: val_loss improved from 0.85203 to 0.82816, saving model to ../CustomVGG_LATEST.h5
595/595 [==============================] - 3180s 5s/step - loss: 0.8665 - accuracy: 0.6772 - val_loss: 0.8282 - val_accuracy: 0.6942
Epoch 2/2
595/595 [==============================] - ETA: 0s - loss: 0.8402 - accuracy: 0.6845
Epoch 2: val_loss improved from 0.82816 to 0.80357, saving model to ../CustomVGG_LATEST.h5
595/595 [==============================] - 3341s 6s/step - loss: 0.8402 - accuracy: 0.6845 - val_loss: 0.8036 - val_accuracy: 0.7094


In [54]:
# Testing after 3 epochs
model_test = model.evaluate(test_dataset)

print("Test loss: ", model_test[0])
print("Test accuracy: ", model_test[1])

186/186 [==============================] - 868s 5s/step - loss: 0.9832 - accuracy: 0.6198
Test loss:  0.9832393527030945
Test accuracy:  0.6197943091392517


In [61]:
# Epoch 5 to 15
model.fit(train_dataset, validation_data=validation_dataset, epochs=10, callbacks=callbacks)

Epoch 1/10
595/595 [==============================] - ETA: 0s - loss: 0.7939 - accuracy: 0.7015
Epoch 1: val_loss improved from 0.83177 to 0.74159, saving model to ../CustomVGG_LATEST.h5
595/595 [==============================] - 3135s 5s/step - loss: 0.7939 - accuracy: 0.7015 - val_loss: 0.7416 - val_accuracy: 0.7268
Epoch 2/10
595/595 [==============================] - ETA: 0s - loss: 0.7654 - accuracy: 0.7130
Epoch 2: val_loss improved from 0.74159 to 0.71801, saving model to ../CustomVGG_LATEST.h5
595/595 [==============================] - 3122s 5s/step - loss: 0.7654 - accuracy: 0.7130 - val_loss: 0.7180 - val_accuracy: 0.7365
Epoch 3/10
595/595 [==============================] - ETA: 0s - loss: 0.7425 - accuracy: 0.7219
Epoch 3: val_loss did not improve from 0.71801
595/595 [==============================] - 3144s 5s/step - loss: 0.7425 - accuracy: 0.7219 - val_loss: 0.7195 - val_accuracy: 0.7284
Epoch 4/10
595/595 [==============================] - ETA: 0s - loss: 0.7118 - accur

In [62]:
# Testing after 15 epochs
model_test = model.evaluate(test_dataset)

print("Test loss: ", model_test[0])
print("Test accuracy: ", model_test[1])

186/186 [==============================] - 844s 5s/step - loss: 1.1215 - accuracy: 0.6370
Test loss:  1.1215119361877441
Test accuracy:  0.6369920969009399


In [63]:
# Testing after 15 epochs
model_test = model.evaluate(test_dataset)

print("Test loss: ", model_test[0])
print("Test accuracy: ", model_test[1])

186/186 [==============================] - 803s 4s/step - loss: 1.1215 - accuracy: 0.6370
Test loss:  1.1215119361877441
Test accuracy:  0.6369920969009399
